In [ ]:
import xml.etree.ElementTree as ET

In [ ]:
icd_10_file = "../data/external/icd10cm-tabular-2023.xml"

In [ ]:
tree = ET.parse(icd_10_file)

In [ ]:
root = tree.getroot()

In [ ]:
chapters = root.findall("chapter")

In [ ]:
chapter = chapters[0]

In [ ]:
sections = chapter.findall('section')

In [ ]:
section = sections[0]

In [ ]:
section.text

In [ ]:
def get_item(diag):
    children =None
    if diag.findall('diag'):
        children = [get_item(x) for x in diag.findall('diag')]
    code = diag.find('name').text
    desc = diag.find('desc').text
    item = {code: desc}
    return [item, children]

In [ ]:
def get_text(diag):
    children =[]
    if diag.findall('diag'):
        for x in diag.findall('diag'):
            children.extend(get_text(x))
    code = diag.find('name').text
    desc = diag.find('desc').text
    # desc = [desc]
    children.append(desc)
    # item = {code: desc}
    return children

In [ ]:
[get_item(x) for x in section.findall('diag')]

In [ ]:
result = []
for chapter in chapters:
    for section in chapter.findall('section'):
        for x in section.findall('diag'):
            y = get_text(x)
            print(y, '\n')
            result.extend(y)


In [ ]:
len(result)

In [ ]:
un_results = list(set(result))

In [ ]:
import fuzzywuzzy
from fuzzywuzzy import process

In [ ]:
un_results = [x.lower().strip() for x in un_results]

In [ ]:
from CTnlp.parsers import parse_clinical_trials_from_folder

In [ ]:
cts = parse_clinical_trials_from_folder("../data/external/ClinicalTrials.2021-04-27.part1", first_n=100000)

In [ ]:
matching_results = []
for ct in cts[:3]:
    for condition in ct.conditions:
        if condition.lower().strip() in un_results:
            matching_results.append([condition, condition, 100])
            print(condition, condition)
        else:
            item, score = process.extractOne(condition, un_results)
            if score > 90:
                print(condition, item, score)
            else:
                print('no', item, score, condition)
            matching_results.append([condition, item, score])

In [ ]:
process.extractOne(condition, un_results)

In [ ]:
condition

In [ ]:
cts[0].exclusion

In [ ]:
smokers / len(cts)

In [ ]:
drinkers / len(cts)

In [ ]:
drinkers

In [ ]:
smokers = 0
drinkers = 0
for ct in cts:
    # print(ct.criteria.find('drink'))
    if ct.criteria.find('drink') != -1:
        # print(ct.criteria[ct.criteria.find('drink')-20 : ct.criteria.find('drink')+25])
        drinkers += 1

    if ct.criteria.find('smok') != -1:
        print(ct.criteria[ct.criteria.find('smok')-20 : ct.criteria.find('smok')+25])
        smokers += 1
    # print(ct.criteria.find('smok'))